In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import math
import tensorflow as tf
from tensorflow.python import keras
from keras.layers import Input, Dense, SimpleRNN, RNN, LSTM
from keras.models import Model
from keras.models import Sequential
from sklearn import preprocessing

In [2]:
csv_files = glob.glob("*.csv")

# 讀取資料
data = {}
for file in csv_files:
    data[os.path.splitext(file)[0]] = pd.read_csv(file)

In [3]:
#set_index    
df_light = data['light_train_source_labels'].set_index('user_id', drop=True)
df_light_y = data['light_train_target_labels'].set_index('user_id', drop=True)
df_light_test = data['light_test_source_labels'].set_index('user_id', drop=True)
sample = data['sample']
df_event = data['train_source_events']
df_event_y = data['train_target_events']
df_event_test = data['test_source_events']
df_reduce = data['reduce_new']
df_event_y_reduce = data['df_event_y_new']
df_event_test_reduce = data['df_event_test_reduce']

In [4]:
df_reduce = df_reduce.drop(['Unnamed: 0', 'time_slot_2', 'event_time', 'until', 'device_id', 'session_id'], axis=1)
df_reduce_group = df_reduce.groupby(['user_id', 'time_slot_1']).mean().reset_index()
scaler = preprocessing.StandardScaler().fit(df_reduce_group)
df_reduce_group_scaled = scaler.transform(df_reduce_group)

In [5]:
df_event_test_reduce = df_event_test_reduce.drop(['Unnamed: 0', 'time_slot_2', 'event_time', 'until', 'device_id', 'session_id'], axis=1)
df_event_test_reduce_group = df_event_test_reduce.groupby(['user_id', 'time_slot_1']).mean().reset_index()
scaler_test = preprocessing.StandardScaler().fit(df_event_test_reduce_group)
df_event_test_reduce_group_scaled = scaler_test.transform(df_event_test_reduce_group)

In [6]:
df_reduce_group_scaled

array([[-1.73106141, -1.5597876 , -0.53876998, ..., -1.03236154,
        -0.34667815, -0.70645212],
       [-1.73106141, -1.43795596, -0.50532367, ..., -1.03236154,
        -0.3885877 , -0.70645212],
       [-1.73106141, -1.07246104,  0.08225198, ..., -1.03236154,
        -0.59813544, -0.70645212],
       ...,
       [ 1.72636289,  0.02402372, -0.57698308, ..., -1.03236154,
        -1.10105002, -0.70645212],
       [ 1.72636289,  0.51135028, -0.57698308, ..., -1.03236154,
        -1.10105002, -0.70645212],
       [ 1.72636289,  0.99867683, -0.64596363, ..., -1.03236154,
        -0.03235653, -0.70645212]])

In [7]:
user = 0
count = 0
time = 0
X_df = []
while user < 30460 or count < 172394:
    if user == 30460 or count == 172394:
        break
    if df_reduce_group['user_id'][count] == user:
        if time == df_reduce_group['time_slot_1'][count]:
            X_df.append(df_reduce_group_scaled[count])
            count += 1
            time += 1
        else:
            X_df.append([0 for zero in range(8)])
            time += 1
    elif time < 28:
        for i in range(28-time):
            X_df.append([0 for zero in range(8)])
            time = 28
    if time == 28:
        time = 0
        user += 1

In [8]:
for i in range(28-time):
    X_df.append([0 for zero in range(8)])

In [9]:
print(user, count, time)

30459 172394 22


In [ ]:
X_df_reshape = pd.DataFrame(X_df)
# X_df_reshape = X_df_reshape.values.reshape(30460, 8, 28)

In [32]:
X_train_total = [[] for i in range(28)]
for i in range(len(X_df_reshape)):
    a = i % 28
    X_train_total[a].append(X_df_reshape.loc[i])

In [11]:

len(df_event_test_reduce_group_scaled)

42715

In [38]:
user_test = 30460
count_test = 0
time_test = 0
X_df_test = []
while user_test < 38076 or count_test < 42715:
    if user_test == 38076 or count_test == 42715:
        break
    if df_event_test_reduce_group['user_id'][count_test] == user_test:
        if time_test == df_event_test_reduce_group['time_slot_1'][count_test]:
            X_df_test.append(df_event_test_reduce_group_scaled[count_test])
            count_test += 1
            time_test += 1
        else:
            X_df_test.append([0 for zero in range(8)])
            time_test += 1
    elif time_test < 28:
        for i in range(28-time_test):
            X_df_test.append([0 for zero in range(8)])
            time_test = 28
    if time_test == 28:
        time_test = 0
        user_test += 1

In [13]:
len(X_df_test)

213242

In [14]:
print(user_test, count_test, time_test)

38075 42715 22


In [39]:
for i in range(28-time_test):
    X_df_test.append([0 for zero in range(8)])

In [16]:
len(X_df_test)

213248

In [40]:
X_df_test_reshape = pd.DataFrame(X_df_test)
# X_df_test_reshape = X_df_test_reshape.values.reshape(7616, 8, 28)

In [41]:
X_test_total = [[] for i in range(28)]
for i in range(len(X_df_test_reshape)):
    a = i % 28
    X_test_total[a].append(X_df_test_reshape.loc[i])

In [45]:
df_light_y = df_light_y.reset_index(drop=True)
df_light_y = df_light_y.astype(int)

In [46]:
df_light_y

time_slot_0  time_slot_1  time_slot_2  time_slot_3  time_slot_4   
0                0            1            0            0            0  \
1                0            0            0            0            0   
2                0            0            0            0            0   
3                1            1            0            0            0   
4                0            0            0            0            1   
...            ...          ...          ...          ...          ...   
30455            0            0            0            0            0   
30456            0            0            1            0            1   
30457            0            0            0            0            1   
30458            0            1            0            1            0   
30459            0            0            0            0            0   

       time_slot_5  time_slot_6  time_slot_7  time_slot_8  time_slot_9  ...   
0                1            0            0            0            1  ...  \
1                0            0            0            1            0  ...   
2                1            0            0            0            1  ...   
3                1            0            0            1            1  ...   
4                0            0            1            0            0  ...   
...            ...          ...          ...          ...          ...  ...   
30455            1            0            0            0            0  ...   
30456            1            0            0            0            0  ...   
30457            1            0            0            0            1  ...   
30458            0            0            0            0            1  ...   
30459            0            0            0            0            0  ...   

       time_slot_18  time_slot_19  time_slot_20  time_slot_21  time_slot_22   
0                 0             0             0             1             0  \
1                 1             0             1             0             0   
2                 1             0             0             0             1   
3                 0             0             0             1             0   
4                 0             1             0             0             0   
...             ...           ...           ...           ...           ...   
30455             0             0             0             0             0   
30456             0             0             0             0             0   
30457             0             0             0             1             0   
30458             0             0             0             0             0   
30459             0             0             0             1             0   

       time_slot_23  time_slot_24  time_slot_25  time_slot_26  time_slot_27  
0                 0             0             0             0             0  
1                 0             1             1             0             0  
2                 0             0             0             0             0  
3                 0             0             1             0             0  
4                 1             1             0             0             0  
...             ...           ...           ...           ...           ...  
30455             0             0             0             0             0  
30456             0             0             0             0             0  
30457             0             0             0             0             0  
30458             0             0             0             0             1  
30459             0             0             0             0             0  

[30460 rows x 28 columns]

In [98]:
df_light_y.columns[0]

'time_slot_0'

In [100]:
for t in range(28):
    X_train = pd.DataFrame(X_train_total[t])
    y_train = pd.DataFrame(df_light_y[df_light_y.columns[t]])
    X_test = pd.DataFrame(X_test_total[t])
    X_train = np.reshape(X_train, (X_train.shape[0], 1,8))
    X_test = np.reshape(X_test, (X_test.shape[0], 1,8))
    model = Sequential()
    model.add(LSTM(4, return_sequences=False, input_shape=(1, 8)))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X_train, y_train, batch_size=128, epochs=30, validation_split=0.1)
    predictions = model.predict(X_test)

    for i in range(len(predictions)):
        if predictions[i] > predictions.mean():
            predictions[i] = 1.0
        else:
            predictions[i] = 0.0
    sample[sample.columns[t + 1]] = predictions

Epoch 1/30
215/215 [==============================] - 3s 5ms/step - loss: 0.6589 - accuracy: 0.7447 - val_loss: 0.6196 - val_accuracy: 0.7643
Epoch 2/30
215/215 [==============================] - 1s 3ms/step - loss: 0.5802 - accuracy: 0.7686 - val_loss: 0.5546 - val_accuracy: 0.7640
Epoch 3/30
215/215 [==============================] - 1s 3ms/step - loss: 0.5340 - accuracy: 0.7685 - val_loss: 0.5298 - val_accuracy: 0.7640
Epoch 4/30
215/215 [==============================] - 1s 2ms/step - loss: 0.5220 - accuracy: 0.7686 - val_loss: 0.5234 - val_accuracy: 0.7636
Epoch 5/30
215/215 [==============================] - 1s 3ms/step - loss: 0.5199 - accuracy: 0.7686 - val_loss: 0.5219 - val_accuracy: 0.7636
Epoch 6/30
215/215 [==============================] - 1s 3ms/step - loss: 0.5195 - accuracy: 0.7685 - val_loss: 0.5211 - val_accuracy: 0.7636
Epoch 7/30
215/215 [==============================] - 1s 4ms/step - loss: 0.5194 - accuracy: 0.7687 - val_loss: 0.5208 - val_accuracy: 0.7636
Epoch 

In [102]:
sample.to_csv('sample.csv')

In [103]:
sample

user_id  time_slot_0  time_slot_1  time_slot_2  time_slot_3   
0       30460          0.0          0.0          0.0          0.0  \
1       30461          1.0          1.0          0.0          0.0   
2       30462          0.0          1.0          0.0          0.0   
3       30463          0.0          0.0          0.0          0.0   
4       30464          0.0          0.0          0.0          0.0   
...       ...          ...          ...          ...          ...   
7611    38071          0.0          0.0          0.0          0.0   
7612    38072          0.0          0.0          0.0          0.0   
7613    38073          0.0          0.0          0.0          0.0   
7614    38074          0.0          0.0          0.0          0.0   
7615    38075          0.0          0.0          0.0          0.0   

      time_slot_4  time_slot_5  time_slot_6  time_slot_7  time_slot_8  ...   
0             0.0          0.0          0.0          1.0          0.0  ...  \
1             0.0          1.0          0.0          0.0          0.0  ...   
2             0.0          1.0          0.0          0.0          0.0  ...   
3             0.0          0.0          0.0          0.0          0.0  ...   
4             1.0          0.0          1.0          0.0          1.0  ...   
...           ...          ...          ...          ...          ...  ...   
7611          0.0          0.0          0.0          0.0          0.0  ...   
7612          0.0          0.0          0.0          0.0          0.0  ...   
7613          0.0          0.0          0.0          0.0          0.0  ...   
7614          0.0          0.0          0.0          0.0          0.0  ...   
7615          0.0          1.0          0.0          0.0          0.0  ...   

      time_slot_18  time_slot_19  time_slot_20  time_slot_21  time_slot_22   
0              0.0           1.0           0.0           0.0           0.0  \
1              0.0           0.0           1.0           1.0           0.0   
2              0.0           1.0           0.0           0.0           0.0   
3              0.0           0.0           0.0           0.0           0.0   
4              0.0           0.0           0.0           0.0           0.0   
...            ...           ...           ...           ...           ...   
7611           0.0           0.0           0.0           0.0           0.0   
7612           0.0           0.0           1.0           1.0           0.0   
7613           0.0           0.0           1.0           1.0           0.0   
7614           0.0           0.0           0.0           0.0           0.0   
7615           0.0           0.0           0.0           0.0           0.0   

      time_slot_23  time_slot_24  time_slot_25  time_slot_26  time_slot_27  
0              1.0           0.0           0.0           0.0           1.0  
1              0.0           0.0           0.0           0.0           0.0  
2              0.0           0.0           0.0           0.0           0.0  
3              0.0           0.0           0.0           0.0           0.0  
4              0.0           0.0           0.0           0.0           0.0  
...            ...           ...           ...           ...           ...  
7611           0.0           0.0           0.0           0.0           0.0  
7612           0.0           1.0           1.0           0.0           0.0  
7613           0.0           1.0           1.0           0.0           0.0  
7614           0.0           0.0           0.0           0.0           0.0  
7615           0.0           0.0           0.0           0.0           0.0  

[7616 rows x 29 columns]

In [105]:
sample.loc[0]

user_id         30460.0
time_slot_0         0.0
time_slot_1         0.0
time_slot_2         0.0
time_slot_3         0.0
time_slot_4         0.0
time_slot_5         0.0
time_slot_6         0.0
time_slot_7         1.0
time_slot_8         0.0
time_slot_9         0.0
time_slot_10        0.0
time_slot_11        1.0
time_slot_12        0.0
time_slot_13        0.0
time_slot_14        0.0
time_slot_15        1.0
time_slot_16        0.0
time_slot_17        0.0
time_slot_18        0.0
time_slot_19        1.0
time_slot_20        0.0
time_slot_21        0.0
time_slot_22        0.0
time_slot_23        1.0
time_slot_24        0.0
time_slot_25        0.0
time_slot_26        0.0
time_slot_27        1.0
Name: 0, dtype: float64

In [107]:
user_reduced = data['df_event_test_group_user']

user_id = 0
for i in range(len(sample)):
    user = sample['user_id'][i]
    if user == user_reduced['user_id'][user_id]:
        user_id += 1
        if user_id == len(user_reduced):
            break
    else:
        sample.loc[i] = [user] + [0.0 for i in range(28) ]

In [110]:
sample.to_csv('sample.csv')